In [18]:
import os
import PyPDF2
import docx
import nltk

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pyfiglet import figlet_format
from termcolor import colored


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

In [26]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

In [30]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

In [32]:
def extract_text_from_docx(docx_path):
    doc = docx.Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])

In [34]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenization
    filtered_tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return " ".join(filtered_tokens)

In [36]:
def rank_resume(job_description, resume_text):
    corpus = [preprocess_text(job_description), preprocess_text(resume_text)]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)
    similarity_score = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
    return similarity_score

In [ ]:
def employer_mode():
    job_role = input("Enter the Job Role: ")
    job_description = input("Enter the Job Description: ")
    with open("job_description.txt", "w", encoding="utf-8") as f:
        f.write(f"Role: {job_role}\n\n{job_description}")
    print("Job description saved successfully!")

In [ ]:
def candidate_mode():
    if not os.path.exists("job_description.txt"):
        print("No job description found! Please ask the employer to create one first.")
        return

    with open("job_description.txt", "r", encoding="utf-8") as f:
        job_description = f.read()
        resume_path = input("Enter File Path: ") 
    if not os.path.exists(resume_path):
        print("File not found! Please check the path and try again.")
        return

    if resume_path.endswith(".pdf"):
        resume_text = extract_text_from_pdf(resume_path)
    elif resume_path.endswith(".docx"):
        resume_text = extract_text_from_docx(resume_path)
    else:
        print("Unsupported file format! Please provide a PDF or DOCX file.")
        return

    score = rank_resume(job_description, resume_text)
    print(f"Your ATS Score: {score:.2f} (Higher is better)")

In [ ]:
def main():
    a = figlet_format("[ Application Tracking System ]", font="slant", width=900)
    print(colored(a, "green"))  # Prints the stylish "ATS" text in green
    print("Welcome to AI-powered ATS System!")
    user_type = input("Are you an Employer or a Candidate? (E/C): ").strip().lower()

    if user_type == "E":
        employer_mode()
    elif user_type == "C":
        candidate_mode()
    else:
        print("Invalid choice! Please restart and enter 'E' for Employer or 'C' for Candidate.")

if __name__ == "__main__":
    main()